In [9]:
import numpy as np
import pandas as pd
import datetime
from numpy.linalg import inv
from pandas_datareader import data as pdr
import yfinance as yfin
#import matplotlib.pyplot as plt
#from scipy.optimize import minimize
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
start = datetime.datetime(2010,1,1)
end = datetime.datetime.now()
yfin.pdr_override()


In [10]:
def data_returns(data): #generating daily avg assets return 
    returns = (data/data.shift(1))-1
    return returns

In [11]:
def portfolio_covariance(data): #generating yearly avg portfolio risk
    yearly_port_cov = data_returns(data).cov()*252
    return yearly_port_cov
    

#### Markowitz Portfolio Theory Function

In [4]:
# 1000 combination with function (Simple return)

def combination(Ticker):

    #load data and create random weights dataframe
    data = pd.DataFrame()
    data = pdr.get_data_yahoo(Ticker,start,end)['Adj Close']
    
    weights_array = pd.DataFrame(columns=[Ticker])

    #empty array for Risk and Return Columns
    port_return = []
    port_risk = []
  
    #generating 1000 random combination for the portfolio
    for w in range (1000):
        #get random weights for each Ticker
        weights = np.random.random(len(Ticker)) 

        #weights return array
        weights = weights/np.sum(weights) 
        
        #insert weights (array) to a dataframe called each_weights
        weights_array.loc[len(weights_array)] = weights.tolist()  
        
        #calculate avg annual return and store it to array port_return
        port_return.append(np.sum(weights * (data_returns(data).mean()*252)))
        
        #calculate avg annual std dev/risk and store it to array port_risk
        port_risk.append(np.sqrt(np.dot(weights.T, np.dot(portfolio_covariance(data), weights)))) 


    #create dataframe called portfolios to store risk and return
    portfolios = pd.DataFrame({'Return':port_return, 'Risk': port_risk}) 
    for i in range(len(Ticker)):   
        weights_array.columns.values[i] = Ticker[i]
        
    result = pd.concat([weights_array, portfolios], axis=1) #concat dataframe each_weights and portfolios
    
    #return port_return, port_risk
    return  result

In [12]:
#Calculation Minimum Variance Portfolio with Lagrange multiplier and Cramer's Rule based on Modul Portfolio Selection from Prof. Dr. Andreas Görg
def Min_Varianz(Ticker):
    
    data = pd.DataFrame()
    data = pdr.get_data_yahoo(Ticker,start,end)['Adj Close']

    #Variable
    Mu = (data_returns(data).mean()*252).to_numpy()
    Sigma = (portfolio_covariance(data)).to_numpy()
    One = [1 for x in range(len(Mu))]
    One_arr = np.array(One)
    Sigma_inverse = inv(Sigma)

    a = np.dot(np.dot(Mu.T,Sigma_inverse),Mu)
    b = np.dot(np.dot(Mu.T,Sigma_inverse),One_arr)
    c = np.dot(np.dot(One_arr,Sigma_inverse),One_arr)


    Return_x = b/c
    Risk_x = 1/np.sqrt(c)

    weights = (np.dot((Return_x*c-b)*Sigma_inverse,Mu)+(np.dot((a-Return_x*b)*Sigma_inverse,One_arr)))/(a*c-b**2)
    
    Min_Port = np.array([Return_x,Risk_x])

    Data = np.concatenate((weights,Min_Port))

    Columns = Ticker
    Columns.append('Return')
    Columns.append('Risk')

    Min_Varianz_Data = pd.DataFrame(columns=Columns)
    Min_Varianz_Data.loc[len(Min_Varianz_Data)] = Data    


    return Min_Varianz_Data

#### Calling Function

In [13]:
xxx = ['AAPL','TSLA','COKE']

In [14]:
my_combination = combination(xxx)

[*********************100%***********************]  3 of 3 completed


In [15]:
my_combination['Sharpe Ratio'] = (my_combination['Return']-0.0304)/my_combination['Risk']
my_combination

,AAPL,TSLA,COKE,Return,Risk,Sharpe Ratio
0,0.400323,0.391693,0.207984,0.321640,0.261001,1.115854
1,0.135826,0.535028,0.329146,0.348238,0.301357,1.054690
2,0.082029,0.860511,0.057460,0.264044,0.305505,0.764778
3,0.497361,0.166550,0.336089,0.364059,0.297199,1.122680
4,0.508982,0.116777,0.374242,0.376037,0.311979,1.107883
...,...,...,...,...,...,...
995,0.174298,0.640111,0.185592,0.306290,0.279321,0.987717
996,0.478335,0.413308,0.108357,0.294475,0.247376,1.067506
997,0.061519,0.346796,0.591684,0.424804,0.385927,1.021967
998,0.397769,0.452122,0.150109,0.304042,0.253734,1.078462


In [16]:
Sharpe_max = my_combination[my_combination['Sharpe Ratio']== my_combination['Sharpe Ratio'].max()]
Sharpe_max

,AAPL,TSLA,COKE,Return,Risk,Sharpe Ratio
456,0.464086,0.267561,0.268352,0.342314,0.274516,1.136231


In [17]:
Min_Var_Portfolio = Min_Varianz(xxx)
Min_Var_Portfolio

[*********************100%***********************]  3 of 3 completed


,AAPL,TSLA,COKE,Return,Risk
0,0.578688,0.383733,0.037579,0.276883,0.243822


In [18]:
Min_Var_Portfolio['Sharpe Ratio'] = (Min_Var_Portfolio['Return']-0.0304)/Min_Var_Portfolio['Risk']
Min_Var_Portfolio


,AAPL,TSLA,COKE,Return,Risk,Sharpe Ratio
0,0.578688,0.383733,0.037579,0.276883,0.243822,1.010915


In [19]:
#More Simple Version of Minimum Variance

# Find index of row with minimum risk
min_risk_idx = my_combination['Risk'].idxmin()

# Subset DataFrame to only include rows with minimum risk
min_risk_df = my_combination[my_combination['Risk'] == my_combination.loc[min_risk_idx, 'Risk']]

# Find index of row with maximum return among rows with minimum risk
optimal_return_idx = min_risk_df['Return'].idxmax()

# Subset DataFrame to only include row with both minimum risk and maximum return
optimal_portfolio = min_risk_df[min_risk_df['Return']==min_risk_df.loc[optimal_return_idx,'Return']]

In [22]:
#reformat data
def reformat_Data(Data):
     
     Data_1 = Data.copy()
     Data_1.iloc[:,:-3] = (Data_1.iloc[:,:-3]*100).round(2).astype(str) + "%"
     Data_1.iloc[:,-3:] = (Data_1.iloc[:,-3:]*100).round(2)
     
     return Data_1


In [23]:

Scatter_Data = reformat_Data(my_combination)
Min_Varianz_Data = reformat_Data(Min_Var_Portfolio)
Sharpe_max_Data = reformat_Data(Sharpe_max)
Optimal_Portfolio_Data = reformat_Data(optimal_portfolio)

In [ ]:
sorted = my_combination.sort_values(by=['Risk'])
test = sorted.groupby(np.arange(len(sorted))//8).max()
test

In [ ]:
sorted.groupby(np.arange(len(sorted))//8)['Sharpe Ratio'].agg(['max'])

#### Visualization

In [ ]:
#Visualizing the 1000 combination data
fig1= px.scatter(data_frame=Scatter_Data,
               x='Risk',y='Return',hover_data=Scatter_Data.columns, color='Sharpe Ratio',
               )

#Visualizing a portfolio with lowest risk
fig2 = px.scatter(data_frame=Min_Varianz_Data, x = 'Risk', y = 'Return',hover_data=Min_Varianz_Data.columns,labels=dict(Risk="Risk (%)", Return="Return (%)"))
#Visualizing a portfolio with highest sharpe ratio
fig3 = px.scatter(data_frame=Sharpe_max_Data, x = 'Risk', y = 'Return',hover_data=Sharpe_max_Data.columns,labels=dict(Risk="Risk (%)", Return="Return (%)"))

fig4 =px.scatter(data_frame=optimal_portfolio,x = 'Risk', y = 'Return',hover_data=optimal_portfolio.columns,labels=dict(Risk="Risk (%)", Return="Return (%)"))

#adding marker to fig2 and fig3
fig2.update_traces(marker=dict(size=15, color="Red",symbol="x"))
fig3.update_traces(marker=dict(size=15, color="Green",symbol="x"))
fig4.update_traces(marker=dict(size=15, color="Blue",symbol="x"))


#here i'm trying to draw a efficient frontier line from the visualization but still not getting the result i wanted
fig5= px.line(data_frame=test,
               x='Risk',y='Return'
               )


fig = go.Figure(data = fig1.data + fig2.data + fig3.data+fig4.data+fig5.data)
fig.update_layout(
    title="Portfolio Combination",
    xaxis_title="Risk (%)",
    yaxis_title="Return (%)",
    font=dict(
        family="Bahnschrift",
        size=18,
        color="RebeccaPurple"
    )
)
fig.show()